In [1]:
import numpy as np
import os
import math
from tqdm import tqdm_notebook

dir_ = './data/'
file_name = 'normalized_filter_track_5_user_100.csv'

In [2]:
train = np.loadtxt(os.path.join(dir_, 'rating_matrix_item_' + file_name), delimiter=',')
test = np.loadtxt(os.path.join(dir_, 'test_list_item_' + file_name), delimiter=',')

In [3]:
np.set_printoptions(suppress=True)
print(train)
print(train.shape)

[[1.59574468 0.         0.         ... 0.         0.         0.        ]
 [0.31914894 0.         2.93103448 ... 0.         0.         0.        ]
 [0.10638298 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(54969, 220)


In [4]:
test

array([[   31.        ,  6104.        ,     0.11210762],
       [  119.        , 38669.        ,     0.06756757],
       [  164.        ,  4597.        ,     0.43103448],
       ...,
       [  183.        ,   971.        ,     0.71428571],
       [  182.        , 44194.        ,     0.14285714],
       [  174.        , 13749.        ,     0.09615385]])

In [5]:
num_tracks = len(train)
num_users = len(train[0])
print(num_users,num_tracks)

220 54969


In [6]:
def pearson_correlation(tid1, tid2):
        result = 0.0
        track1_data = train[tid1]
        track2_data = train[tid2]

        rx_avg = item_average_rating(track1_data)
        ry_avg = item_average_rating(track2_data)
        sxy = common_items(track1_data, track2_data)
        
        top_result = 0.0
        bottom_left_result = 0.0
        bottom_right_result = 0.0
        for user in sxy:
            rxs = track1_data[user]
            rys = track2_data[user]

            top_result += (rxs - rx_avg)*(rys - ry_avg)
            bottom_left_result += pow((rxs - rx_avg), 2)
            bottom_right_result += pow((rys - ry_avg), 2)
        bottom_left_result = math.sqrt(bottom_left_result)
        bottom_right_result = math.sqrt(bottom_right_result)
        
        ################################################################
        if (bottom_left_result * bottom_right_result) == 0:
            return -2 # dump the data
        ################################################################
        
        result = top_result/(bottom_left_result * bottom_right_result)
        return result

def item_average_rating(track):
    avg_rating = 0.0
    for i in track:
        avg_rating += i
    avg_rating /= len(track) * 1.0
    return avg_rating

def common_items(i1, i2):
    result = []
    for i in range(num_users):
        if i1[i] > 0 and i2[i] > 0:
            result.append(i)
    return result

In [7]:
def knn(tid, k):
    neighbors = []
    result = []
    for track in range(num_tracks):
        if tid == track:
            continue
        upc = pearson_correlation(tid, track)
        neighbors.append([track, upc])
    sorted_neighbors = sorted(neighbors, key=lambda neighbors: (neighbors[1], neighbors[0]), reverse=True)   # - for desc sort

    for i in range(k):
        if i >= len(sorted_neighbors):
            break
        result.append(sorted_neighbors[i])
    return result

In [8]:
def predict_by_knn(uid, tid, k_nearest_neighbors):
        valid_neighbors = check_neighbors_validattion(uid, k_nearest_neighbors)
        ##################################
        if not len(valid_neighbors):
            return -1
        ##################################
        top_result = 0.0
        bottom_result = 0.0
        for neighbor in valid_neighbors:
            neighbor_id = neighbor[0]
            neighbor_similarity = neighbor[1]   # Wi1
            rating = train[neighbor_id][uid] # rating i,item
            top_result += neighbor_similarity * rating
            bottom_result += neighbor_similarity
        result = top_result/bottom_result
        return result

def check_neighbors_validattion(uid, k_nearest_neighbors):
    result = []
    for neighbor in k_nearest_neighbors:
        neighbor_id = neighbor[0]
        if train[neighbor_id][uid] > 0:
            result.append(neighbor)
    return result


In [9]:
def predict(uid, tid, k):
    
    if len(knn_list[tid]) == 0:
        k_nearest_neighbors = knn(tid, k)
        knn_list[tid] = k_nearest_neighbors
    else:
        k_nearest_neighbors = knn_list[tid]

    prediction = predict_by_knn(uid, tid, k_nearest_neighbors)
    return prediction

In [10]:
# # test data

# test = [
#     [5,5,0,0],
#     [4,5,1,4],
#     [5,4,2,3],
#     [1,5,0,3],
#     [2,3,5,4]
# ]

# num_users = 4
# num_tracks = 5

# temp = train
# train = test

In [11]:
knn_list = []

for i in range(0, num_tracks):
    knn_list.append([])


In [ ]:
k = 30
prediction_list = []

for i in tqdm_notebook(range(len(test))):
    
    target_user = int(test[i][0])
    target_item = int(test[i][1])
    
    prediction = predict(target_user, target_item, k)
    prediction_list.append(prediction)

    print(test[i][0],test[i][1],test[i][2],prediction)

31.0 6104.0 0.11210762331838565 -1
119.0 38669.0 0.06756756756756757 0.33783783783783783
164.0 4597.0 0.43103448275862066 0.04310344827586207
88.0 712.0 1.2671232876712328 -1
121.0 14964.0 1.25 -1
116.0 7128.0 0.16393442622950818 -1
63.0 4096.0 0.043859649122807015 -1
88.0 16055.0 0.136986301369863 0.28253424657534243
185.0 2731.0 0.2631578947368421 -1
63.0 19500.0 0.08771929824561403 -1
80.0 7036.0 0.2702702702702703 0.13513513513513514
124.0 4.0 0.11235955056179775 0.49157303370786515
130.0 21567.0 0.24390243902439024 0.12195121951219512
164.0 9005.0 0.43103448275862066 0.04310344827586207
134.0 300.0 0.08287292817679558 0.16574585635359115
194.0 12843.0 0.7142857142857143 0.23809523809523808
170.0 52620.0 0.5747126436781609 0.05747126436781609
134.0 44112.0 0.027624309392265192 0.046040515653775316
130.0 17281.0 0.12195121951219512 -1
83.0 43558.0 0.09803921568627451 -1
74.0 11972.0 0.19230769230769232 -1
207.0 16745.0 0.0034317089910775567 0.3345916266300618
147.0 7735.0 0.08196721

125.0 35102.0 0.08771929824561403 -1
196.0 47526.0 0.375 0.21428571428571427
209.0 15649.0 2.8 -1
143.0 36045.0 0.0684931506849315 -1
127.0 13469.0 0.35714285714285715 -1
22.0 36619.0 0.2631578947368421 -1
188.0 26455.0 0.3191489361702128 0.3723404255319149
65.0 2216.0 0.4166666666666667 -1
82.0 5261.0 0.11363636363636363 -1
105.0 47830.0 0.05263157894736842 -1
171.0 23711.0 0.10204081632653061 0.06802721088435375
80.0 13416.0 0.5405405405405406 -1
119.0 3284.0 0.9459459459459459 -1
155.0 3593.0 0.1388888888888889 0.4166666666666667
42.0 2329.0 2.3480662983425415 -1
4.0 10396.0 0.14925373134328357 -1
151.0 11798.0 1.4583333333333333 0.125
166.0 41957.0 0.1282051282051282 0.1282051282051282
11.0 27393.0 0.10204081632653061 -1
66.0 14332.0 0.746268656716418 -1
134.0 31435.0 0.027624309392265192 -1
134.0 15388.0 0.027624309392265192 0.04143646408839779
124.0 37395.0 0.2808988764044944 -1
103.0 4984.0 0.23391812865497075 -1
164.0 39998.0 0.04310344827586207 0.04310344827586207
196.0 11495.

167.0 17965.0 0.14354066985645933 0.03289473684210526
101.0 13516.0 0.09259259259259259 -1
199.0 12327.0 0.2830188679245283 0.09433962264150944
4.0 14792.0 0.07462686567164178 -1
191.0 26101.0 0.4166666666666667 -1
24.0 6433.0 0.35714285714285715 -1
130.0 6792.0 0.12195121951219512 -1
179.0 7610.0 0.29069767441860467 0.05813953488372093
68.0 8083.0 0.06172839506172839 -1
116.0 11295.0 0.06147540983606557 -1
6.0 13587.0 0.034013605442176874 -1
54.0 36954.0 0.14705882352941177 -1
52.0 6904.0 1.3173652694610778 -1
22.0 1760.0 0.2631578947368421 -1
7.0 24067.0 0.1724137931034483 -1
115.0 29714.0 0.02631578947368421 -1
174.0 10902.0 0.09615384615384616 0.09615384615384616
167.0 10560.0 0.09569377990430622 0.06624953993375048
41.0 4656.0 0.03571428571428571 -1
7.0 3253.0 0.5172413793103449 -1
124.0 31929.0 0.11235955056179775 -1
112.0 25262.0 0.5 -1
155.0 43115.0 1.1111111111111112 0.9259259259259259
100.0 30772.0 0.10204081632653061 -1
105.0 38908.0 0.15789473684210525 -1
96.0 35911.0 0.294

7.0 22777.0 0.5172413793103449 -1
59.0 415.0 0.23076923076923078 -1
115.0 51771.0 0.47368421052631576 -1
61.0 18889.0 0.22727272727272727 -1
173.0 33873.0 1.0493827160493827 -1
89.0 13989.0 0.08620689655172414 -1
204.0 43158.0 0.17857142857142858 -1
60.0 43785.0 0.6451612903225806 -1
194.0 47151.0 0.23809523809523808 0.23809523809523808
61.0 23596.0 0.15151515151515152 -1
11.0 7358.0 0.10204081632653061 -1
46.0 248.0 0.02262443438914027 -1
37.0 11909.0 0.9615384615384616 -1
182.0 23058.0 0.07142857142857142 0.1607142857142857
185.0 28188.0 0.15037593984962405 -1
74.0 46614.0 0.19230769230769232 -1
82.0 33772.0 0.11363636363636363 -1
47.0 27229.0 0.0625 -1
68.0 19568.0 0.06172839506172839 0.18518518518518517
215.0 82.0 0.14285714285714285 2.5714285714285716
24.0 37351.0 0.17857142857142858 -1
76.0 26557.0 0.16666666666666666 -1
72.0 15147.0 0.14084507042253522 -1
193.0 38837.0 1.588235294117647 0.11764705882352941
55.0 48218.0 0.5172413793103449 -1
7.0 24475.0 0.1724137931034483 -1
63.0

37.0 4866.0 0.9615384615384616 -1
110.0 50035.0 0.07936507936507936 -1
132.0 21029.0 0.49019607843137253 -1
202.0 6142.0 0.18518518518518517 -1
173.0 22538.0 0.06172839506172839 -1
32.0 39462.0 0.21739130434782608 -1
11.0 19026.0 0.30612244897959184 -1
164.0 12540.0 0.04310344827586207 0.43103448275862066
33.0 39741.0 0.07731958762886598 -1
157.0 28223.0 0.08928571428571429 -1
104.0 5706.0 0.16129032258064516 -1
33.0 26662.0 0.07731958762886598 -1
182.0 35216.0 0.03571428571428571 0.11160714285714285
7.0 22085.0 0.8620689655172413 -1
72.0 27709.0 0.2112676056338028 -1
151.0 5651.0 0.4166666666666667 0.15625
88.0 4040.0 0.03424657534246575 -1
88.0 4.0 0.17123287671232876 -1
60.0 36494.0 0.3225806451612903 -1
182.0 50208.0 0.03571428571428571 0.1464285714285714
211.0 35217.0 0.20833333333333334 0.4861111111111111
171.0 8122.0 0.034013605442176874 0.10204081632653061
88.0 41564.0 0.10273972602739725 -1
17.0 10128.0 0.11904761904761904 -1
107.0 38816.0 0.03571428571428571 -1
66.0 47577.0 1

...

167.0 1861.0 0.09569377990430622 0.059808612440191394
106.0 2723.0 0.3472222222222222 -1
134.0 35639.0 0.11049723756906077 -1
124.0 26752.0 0.11235955056179775 -1
59.0 12745.0 0.07692307692307693 -1
73.0 47555.0 0.2127659574468085 -1
10.0 162.0 1.4285714285714286 -1
64.0 12464.0 0.13636363636363635 -1
124.0 13519.0 0.056179775280898875 -1
61.0 16340.0 0.07575757575757576 -1
7.0 8623.0 1.5517241379310345 -1
148.0 22376.0 0.054945054945054944 0.15905147484094853
72.0 14856.0 0.07042253521126761 -1
194.0 51741.0 0.23809523809523808 0.23809523809523808
17.0 2332.0 0.11904761904761904 -1
6.0 17442.0 0.034013605442176874 -1
127.0 42870.0 0.17857142857142858 -1
50.0 18967.0 1.4285714285714286 -1
66.0 49968.0 0.8955223880597015 -1
105.0 29661.0 0.05263157894736842 -1
80.0 50873.0 0.13513513513513514 -1
151.0 48582.0 0.3125 0.20833333333333331
196.0 27952.0 0.5 0.2
176.0 4529.0 0.4166666666666667 0.4166666666666667
189.0 3956.0 0.22435897435897437 0.44871794871794873
128.0 38037.0 0.83333333333

157.0 43910.0 0.08928571428571429 0.08928571428571429
178.0 11332.0 0.055865921787709494 0.027932960893854747
23.0 23972.0 0.07462686567164178 -1
99.0 8008.0 0.18518518518518517 -1
218.0 20087.0 0.05328596802841918 -1
68.0 23156.0 0.06172839506172839 -1
174.0 48104.0 0.4807692307692308 0.09615384615384616
123.0 7905.0 0.19607843137254902 -1
179.0 36106.0 0.05813953488372093 0.11627906976744186
165.0 42368.0 0.07042253521126761 0.3873239436619718
176.0 17160.0 0.8333333333333334 0.20833333333333334
105.0 41338.0 0.05263157894736842 -1
199.0 44677.0 0.04716981132075472 0.09433962264150944
59.0 23031.0 0.07692307692307693 -1
89.0 33284.0 0.1724137931034483 -1
41.0 14296.0 0.39285714285714285 -1
4.0 15159.0 0.14925373134328357 -1
105.0 4820.0 0.05263157894736842 -1
52.0 43184.0 0.059880239520958084 -1
116.0 13798.0 0.08196721311475409 -1
106.0 46335.0 1.4583333333333333 -1
103.0 21020.0 0.3508771929824561 -1
80.0 34150.0 0.2702702702702703 -1
33.0 4357.0 0.10309278350515463 -1
188.0 32205.

38.0 1267.0 0.0423728813559322 -1
61.0 7959.0 0.15151515151515152 -1
41.0 27267.0 0.03571428571428571 -1
143.0 1572.0 0.136986301369863 -1
66.0 9845.0 0.746268656716418 0.14925373134328357
79.0 1418.0 0.05952380952380952 -1
73.0 1196.0 0.10638297872340426 -1
51.0 4108.0 1.25 -1
188.0 22796.0 0.3191489361702128 0.35460992907801425
170.0 11710.0 0.45977011494252873 -1
33.0 6427.0 0.28350515463917525 -1
148.0 45555.0 0.21978021978021978 0.18481518481518486
43.0 45065.0 0.14084507042253522 -1
151.0 12747.0 0.20833333333333334 0.16666666666666666
28.0 34395.0 0.5555555555555556 -1
169.0 1418.0 0.10273972602739725 0.03424657534246575
17.0 32239.0 0.35714285714285715 -1
140.0 11616.0 0.5208333333333334 0.15625
0.0 1886.0 0.2127659574468085 -1
189.0 2919.0 0.32051282051282054 0.2403846153846154
167.0 5006.0 0.023923444976076555 0.04485645933014355
80.0 37332.0 0.13513513513513514 -1
113.0 41801.0 0.08928571428571429 -1
100.0 25999.0 0.10204081632653061 -1
4.0 13790.0 0.07462686567164178 -1
187

179.0 19037.0 0.05813953488372093 -1
134.0 39986.0 0.055248618784530384 -1
71.0 51050.0 0.09259259259259259 -1
102.0 19873.0 0.2 -1
140.0 15733.0 0.20833333333333334 0.625
4.0 3687.0 0.07462686567164178 -1
72.0 12110.0 0.07042253521126761 -1
109.0 28337.0 0.20833333333333334 -1
30.0 17794.0 0.5 -1
43.0 17448.0 0.49295774647887325 -1
7.0 13398.0 0.1724137931034483 0.1724137931034483
7.0 12852.0 0.5172413793103449 -1
110.0 5565.0 0.23809523809523808 -1
148.0 5109.0 0.054945054945054944 0.1041064198958936
148.0 33037.0 0.054945054945054944 0.12820512820512822
212.0 21799.0 0.25 0.125
19.0 11174.0 0.08771929824561403 -1
14.0 2644.0 0.1724137931034483 -1
148.0 43528.0 0.16483516483516483 0.11250654107796967
9.0 25443.0 0.3125 -1
40.0 22996.0 0.19230769230769232 -1
167.0 40370.0 0.04784688995215311 0.04518872939925573
188.0 27516.0 0.10638297872340426 0.5141843971631206
194.0 35573.0 0.23809523809523808 0.23809523809523808
7.0 12383.0 0.5172413793103449 0.25862068965517243
211.0 25582.0 0.41

194.0 52203.0 0.23809523809523808 0.23809523809523808
125.0 53457.0 0.08771929824561403 -1
179.0 34520.0 0.23255813953488372 -1
39.0 9921.0 0.08928571428571429 -1
3.0 5668.0 0.2892561983471074 -1
6.0 12947.0 1.4965986394557824 -1
178.0 9993.0 0.19553072625698323 0.027932960893854747
182.0 33853.0 0.10714285714285714 0.1642857142857143
160.0 28592.0 1.8461538461538463 -1
171.0 38720.0 0.23809523809523808 0.06802721088435375
213.0 29416.0 0.14285714285714285 0.07142857142857142
4.0 144.0 0.07462686567164178 -1
116.0 1273.0 0.08196721311475409 -1
198.0 34434.0 0.7692307692307693 0.7692307692307692
50.0 9590.0 1.6326530612244898 -1
121.0 756.0 1.25 -1
124.0 34776.0 0.056179775280898875 -1
167.0 3274.0 0.07177033492822966 0.0518341307814992
182.0 35432.0 0.03571428571428571 0.07142857142857142
179.0 53027.0 0.05813953488372093 0.29069767441860467
99.0 17767.0 0.37037037037037035 -1
210.0 850.0 0.14084507042253522 0.2640845070422535
110.0 23344.0 0.5555555555555556 -1
142.0 52355.0 0.0694444

In [ ]:
np.savetxt(os.path.join(dir_, 'prediction__item_' + file_name), prediction_list, delimiter=",")

In [16]:
print(len(test)/8)

17559.125
